In [1]:
import os
import torch
import transformers

HF_TOKEN = os.getenv("HF_TOKEN")

# model_name = "meta-llama/Llama-3.2-1B"
# model_name = "meta-llama/Llama-3.2-1B-Instruct"
# model_name = "meta-llama/Llama-3.2-3B"
model_name = "meta-llama/Llama-3.2-3B-Instruct"
# model_name = "google/gemma-2-2b"
# model_name = "google/gemma-2-2b-it"
# model_name = "google/gemma-2-9b"
# model_name = "google/gemma-2-9b-it"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    token=HF_TOKEN,
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    token=HF_TOKEN,
)

print(model, model.config)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [2]:
from src.util.json_io import *

train_qnas = load_jsonlines(f'data/gsm8k/train.jsonl')
# train_qnas = load_jsonlines(f'data/gsm8k/train_socratic.jsonl')
test_qnas = load_jsonlines(f'data/gsm8k/test.jsonl')
# test_qnas = load_jsonlines(f'data/gsm8k/test_socratic.jsonl')
len(train_qnas), len(test_qnas)

(7473, 1319)

In [3]:
import random; rseed = 42; random.seed(rseed)

nshot_prompt = f""
for i in random.sample(range(len(train_qnas)), 8):
    nshot_prompt += f"Question: {train_qnas[i]['question']}\nAnswer: {train_qnas[i]['answer']}\n\n"

print(nshot_prompt)

Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?
Answer: There are 144/12 = <<144/12=12>>12 sets of 12 cans that the family collected.
So, the family would receive $0.50 x 12 = $<<0.50*12=6>>6 for the cans.
There are 20/5 = <<20/5=4>>4 sets of 5 kilograms of newspapers that the family collected.
So, the family would receive $1.50 x 4 = $<<1.50*4=6>>6 for the newspapers.
Therefore, the family would receive a total of $6 + $6 = $<<6+6=12>>12.
#### 12

Question: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?
Answer: Matthew picked 16 + 20 = <<16+20=36>>36 strawberries.
Natalie picked 3

In [4]:
def question_to_prompt(question):
    return f"{nshot_prompt}Question: {question} Let's think step by step.\nAnswer: "

sample_i = 4
print(question_to_prompt(test_qnas[sample_i]['question']))

from src.util.gsm8k_helper import *
test_qnas[sample_i]['answer'], extract_num_from_ans(test_qnas[sample_i]['answer'])

Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?
Answer: There are 144/12 = <<144/12=12>>12 sets of 12 cans that the family collected.
So, the family would receive $0.50 x 12 = $<<0.50*12=6>>6 for the cans.
There are 20/5 = <<20/5=4>>4 sets of 5 kilograms of newspapers that the family collected.
So, the family would receive $1.50 x 4 = $<<1.50*4=6>>6 for the newspapers.
Therefore, the family would receive a total of $6 + $6 = $<<6+6=12>>12.
#### 12

Question: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?
Answer: Matthew picked 16 + 20 = <<16+20=36>>36 strawberries.
Natalie picked 3

('If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.\nIf she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-15-25=<<60-15-25=20>>20 cups of chicken feed.\n#### 20',
 20)

In [5]:
def generate_answer(input_text):

    input_ids = tokenizer.encode(
        input_text, 
        return_tensors='pt', 
    ).to(model.device)

    with torch.no_grad():
        generated_tokens = model.generate(
            input_ids,
            max_length=input_ids.shape[1] + 512,
            do_sample=True, top_k=1, # greedy decoding (deterministic)
            eos_token_id=tokenizer.encode(text='\n\n', add_special_tokens=False)[0],
            pad_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(generated_tokens[0])

    return generated_text.split('Answer: ')[-1].split(tokenizer.special_tokens_map['eos_token'])[0].split('\n\n')[0]

print('=== Input Text ===\n', question_to_prompt(test_qnas[sample_i]['question']))
print('\n=== Ground Truth ===\n', test_qnas[sample_i]['answer'])
print('\n=== Generated ===\n', generate_answer(question_to_prompt(test_qnas[sample_i]['question'])))

=== Input Text ===
 Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?
Answer: There are 144/12 = <<144/12=12>>12 sets of 12 cans that the family collected.
So, the family would receive $0.50 x 12 = $<<0.50*12=6>>6 for the cans.
There are 20/5 = <<20/5=4>>4 sets of 5 kilograms of newspapers that the family collected.
So, the family would receive $1.50 x 4 = $<<1.50*4=6>>6 for the newspapers.
Therefore, the family would receive a total of $6 + $6 = $<<6+6=12>>12.
#### 12

Question: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?
Answer: Matthew picked 16 + 20 = <<16+20=36>>36 strawberri

c:\Users\sewoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



=== Generated ===
 15 cups of feed are given to the chickens in the morning.
15 cups of feed are given to the chickens in the afternoon.
15 + 25 = <<15+25=40>>40 cups of feed are given to the chickens in the morning and afternoon.
Wendi has 20 chickens, and she gives each chicken 3 cups of feed per day.
20 chickens * 3 cups of feed per chicken = 60 cups of feed per day.
Wendi has already given the chickens 40 cups of feed.
60 - 40 = <<60-40=20>>20 cups of feed are left for the final meal of the day.
#### 20


In [6]:
from datetime import datetime

current_time = datetime.now().strftime("%y%m%d_%H%M%S")
errors_log_path = f"log/errors-{model_name.replace('/', '-')}-{current_time}.txt"
with open(errors_log_path, 'w', encoding='utf-8') as log_file:
    log_file.write(f"{model}\n\n")
    log_file.write(f"{model.config}\n\n")
    log_file.write(f"[random seed]: {rseed}\n\n")
    log_file.write(f"[Nshot prompt]:\n{nshot_prompt}\n\n")
    log_file.write("=" * 40 + "\n\n")

correct_count = total_count = 0
for i, qna in enumerate(test_qnas[:10]):
    input_text = question_to_prompt(qna['question'])

    generated_answer = generate_answer(input_text)
    generated_answer_int = extract_num_from_ans(generated_answer)
    ground_truth_int = extract_num_from_ans(qna['answer'])

    total_count += 1
    if generated_answer_int == ground_truth_int:
        correct_count += 1
        print('o', end='')
    else:
        print('x', end='')
        with open(errors_log_path, 'a', encoding='utf-8') as log_file:
            log_file.write(f"[Question (i:{i})]\n{qna['question']}\n\n")
            log_file.write(f"[Response {model_name}]\n{generated_answer}\n\n")
            log_file.write(f"[Ground Truth]\n{qna['answer']}\n\n")
            log_file.write(f"[Accuracy]\n{correct_count / total_count * 100:.2f}\n\n\n")
print(f"\nAccuracy: {correct_count / total_count * 100:.2f}%")

print(f"Errors logged to {errors_log_path}")

ooxooxoxxo
Accuracy: 60.00%
Errors logged to log/errors-meta-llama-Llama-3.2-3B-Instruct-241104_220836.txt
